In [1]:
import pandas as pd
import numpy as np
from collections import Counter

bg_data = pd.read_csv('data/boardgame_data.csv')
bg_data = bg_data.drop('Unnamed: 0', axis=1)
ind = []
bg_data['Year Published'] = bg_data['Year Published'].apply(lambda x: x.strip('\(\)'))
for i in xrange(len(bg_data)):
    try:
        int(bg_data['Year Published'][i])
    except ValueError:
        ind.append(i)
for i in ind:
    try:
        bg_data.set_value(i, 'Title', bg_data['Title'][i]+' '+bg_data['Year Published'][i])
        bg_data.set_value(i, 'Year Published', 0)
    except TypeError:
        bg_data.set_value(i, 'Title', bg_data['Year Published'][i])
        bg_data.set_value(i, 'Year Published', 0)
bg_data['Year Published'] = pd.to_numeric(bg_data['Year Published'], errors='ignore')
bg_data['Year Published'] = bg_data['Year Published'].astype(int)
bg_data['Min Playing Time'] = bg_data['Play Time'].apply(lambda x: x.split(',')[0].strip('[]'))
bg_data['Max Playing Time'] = bg_data['Play Time'].apply(lambda x: x.split(',')[1].strip('[]'))
bg_data = bg_data.drop('Play Time', axis=1)
bg_data['Min Playing Time'] = pd.to_numeric(bg_data['Min Playing Time'], errors='coerce')
bg_data['Max Playing Time'] = pd.to_numeric(bg_data['Max Playing Time'], errors='coerce')
bg_data['Min Playing Time'] = bg_data['Min Playing Time'].fillna(0)
bg_data['Max Playing Time'] = bg_data['Max Playing Time'].fillna(0)
bg_data['Min Playing Time'] = bg_data['Min Playing Time'].astype(int)
bg_data['Max Playing Time'] = bg_data['Max Playing Time'].astype(int)
bg_data['Age'] = pd.to_numeric(bg_data['Age'], errors='coerce')
bg_data['Age'] = bg_data['Age'].fillna(bg_data['Age'].mean())
bg_data['Age'] = bg_data['Age'].astype(int)
bg_data['Complexity'] = bg_data['Complexity'].astype(float)
bg_data['Min Players'] = pd.to_numeric(bg_data['Min Players'], errors='coerce')
bg_data['Min Players'] = bg_data['Min Players'].fillna(bg_data['Min Players'].mean())
bg_data['Min Players'] = bg_data['Min Players'].astype(int)
bg_data['Max Players'] = pd.to_numeric(bg_data['Max Players'], errors='coerce')
bg_data['Max Players'] = bg_data['Max Players'].fillna(bg_data['Max Players'].mean())
bg_data['Max Players'] = bg_data['Max Players'].astype(int)
bg_data['Min Best Players'] = bg_data['Best Players'].apply(lambda x: x.split('\xe2\x80\x93')[0].strip('+'))
bg_data['Min Best Players'] = pd.to_numeric(bg_data['Min Best Players'], errors='coerce')
bg_data['Min Best Players'] = bg_data['Min Best Players'].fillna(bg_data['Min Best Players'].mean())
bg_data['Min Best Players'] = bg_data['Min Best Players'].astype(int)
bg_data['Max Best Players'] = bg_data['Best Players']
for i in xrange(len(bg_data)):
    bg_data.set_value(i, 'Designers', bg_data.loc[i, 'Designers'].strip('[]').split(', '))
    bg_data.set_value(i, 'Artists', bg_data.loc[i, 'Artists'].strip('[]').split(', '))
    bg_data.set_value(i, 'Publishers', bg_data.loc[i, 'Publishers'].strip('[]').split(', '))
    bg_data.set_value(i, 'Categories', bg_data.loc[i, 'Categories'].strip('[]').split(', '))
    bg_data.set_value(i, 'Mechanisms', bg_data.loc[i, 'Mechanisms'].strip('[]').split(', '))
    try:
        bg_data.set_value(i, 'Max Best Players', bg_data['Best Players'][i].split('\xe2\x80\x93')[1].strip('+'))
    except IndexError:
        bg_data.set_value(i, 'Max Best Players', bg_data['Best Players'][i].split('\xe2\x80\x93')[0].strip('+'))
bg_data['Max Best Players'] = pd.to_numeric(bg_data['Max Best Players'], errors='coerce')
bg_data['Max Best Players'] = bg_data['Max Best Players'].fillna(bg_data['Max Best Players'].mean())
bg_data['Max Best Players'] = bg_data['Max Best Players'].astype(int)
bg_data = bg_data.drop('Best Players', axis=1)

bg_ranking_data = pd.read_csv('data/game_rankings.csv')
bg_ranking_data['Year Published'] = bg_ranking_data['Year Published'].fillna(0)
bg_ranking_data['Year Published'] = bg_ranking_data['Year Published'].astype(int)
bg_ranking_data['Title'] = bg_ranking_data['Title'].apply(lambda x: x.strip())
full_dataset_inner = pd.merge(bg_data, bg_ranking_data, how='inner', on=['Title', 'Year Published'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)
full_dataset_inner.drop(2220, inplace=True)
full_dataset_inner.drop(2221, inplace=True)
full_dataset_inner.drop(2518, inplace=True)
full_dataset_inner.drop(2519, inplace=True)
full_dataset_inner.drop(6568, inplace=True)
full_dataset_inner.drop(6569, inplace=True)

full_dataset = pd.merge(bg_data, bg_ranking_data, how='outer', on=['Title', 'Year Published'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)
full_dataset.drop(2582, inplace=True)
full_dataset.drop(2268, inplace=True)
full_dataset.drop(2269, inplace=True)
full_dataset.drop(2581, inplace=True)
full_dataset.drop(6735, inplace=True)
full_dataset.drop(6736, inplace=True)

In [2]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13612 entries, 0 to 13617
Data columns (total 19 columns):
Title               13612 non-null object
Year Published      13612 non-null int64
Min Players         13502 non-null float64
Max Players         13502 non-null float64
Age                 13502 non-null float64
Complexity          13502 non-null float64
Designers           13502 non-null object
Artists             13502 non-null object
Publishers          13502 non-null object
Categories          13502 non-null object
Mechanisms          13502 non-null object
Min Playing Time    13502 non-null float64
Max Playing Time    13502 non-null float64
Min Best Players    13502 non-null float64
Max Best Players    13502 non-null float64
Rank                13400 non-null float64
Geek Rating         13400 non-null float64
Avg Rating          13400 non-null float64
Num Ratings         13400 non-null float64
dtypes: float64(12), int64(1), object(6)
memory usage: 2.1+ MB


In [3]:
full_dataset_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13290 entries, 0 to 13295
Data columns (total 19 columns):
Title               13290 non-null object
Year Published      13290 non-null int64
Min Players         13290 non-null int64
Max Players         13290 non-null int64
Age                 13290 non-null int64
Complexity          13290 non-null float64
Designers           13290 non-null object
Artists             13290 non-null object
Publishers          13290 non-null object
Categories          13290 non-null object
Mechanisms          13290 non-null object
Min Playing Time    13290 non-null int64
Max Playing Time    13290 non-null int64
Min Best Players    13290 non-null int64
Max Best Players    13290 non-null int64
Rank                13290 non-null int64
Geek Rating         13290 non-null float64
Avg Rating          13290 non-null float64
Num Ratings         13290 non-null int64
dtypes: float64(3), int64(10), object(6)
memory usage: 2.0+ MB
